# # W3P2 - PART2 - city_bikes

assignment file part 1 

----
# CityBikes

In [1]:
import requests
import pandas as pd
import json

Starting with a broad search of all of the available cities:
According to the [citybik.es](https://citybik.es/) home page, there are only 6 Canadian cities to choose from:  

<img src=../images/citybik.es%20CA%20and%20US%20cities.png alt=../images/citybik.es%20CA%20and%20US%20cities.png width="800"/>


Test queries on the API:

In [38]:
ctyBks = requests.get('https://api.citybik.es/v2/networks').json()

In [40]:
ctyBks['networks'][100]

{'company': ['BCycle, LLC'],
 'gbfs_href': 'https://gbfs.bcycle.com/bcycle_bublr/gbfs.json',
 'href': '/v2/networks/bublr-bikes',
 'id': 'bublr-bikes',
 'location': {'city': 'Milwaukee, WI',
  'country': 'US',
  'latitude': 43.0369,
  'longitude': -87.89667},
 'name': 'Bublr Bikes'}

In [58]:
# Number of total Networks:
print(len(ctyBks['networks']))

num_of_networks = len(ctyBks['networks'])

614


_Test country and city locations in json file:_

In [59]:
ctyBks['networks'][0]['location']['country']

'RU'

_Make sure my df-building plan is really a thing using simple example first:_

In [52]:
test1 = [{'a': 100, 'b': 100}]
test2 = [{'a': 220, 'b': 220}]
test3 = [{'a': 330, 'b': 330}]
fulltest = []

fulltest.extend(test1)
fulltest.extend(test2)
fulltest.extend(test3)

print(fulltest)
testdf = pd.DataFrame(fulltest)
testdf

[{'a': 100, 'b': 100}, {'a': 220, 'b': 220}, {'a': 330, 'b': 330}]


,a,b
0,100,100
1,220,220
2,330,330


In [110]:
ctyBks_services = []
for i in range(num_of_networks): 
    city = ctyBks['networks'][i]['location']['city']
    country = ctyBks['networks'][i]['location']['country']
    id = ctyBks['networks'][i]['id']
    
    ctyBks_services.extend([{'city': city, 'country': country, 'id': id}])

ctyBksServiceSites = pd.DataFrame(ctyBks_services)
ctyBksServiceSites.head(10)

,city,country,id
0,Moscow,RU,velobike-moscow
1,Bærum,NO,baerum-bysykkel
2,Siena,IT,bicincitta-siena
3,Maroussi,GR,cyclopolis-maroussi
4,Slough,GB,cycle-hire-slough
5,Maroussi,GR,cyclopolis-nafplio
6,Parco dei Colli di Bergamo,IT,bicincitta-parco-dei-colli-di-bergamo
7,Maroussi,GR,cyclopolis-aigialeia
8,Marathon,GR,cyclopolis-marathon
9,Mantova,IT,bicincitta-mantova


In [71]:
ctyBks_allCCodes = ctyBksServiceSites[['country', 'city']].groupby(['country']).count().sort_values('city', ascending=False)
ctyBks_allCCodes.head(10)

,city
country,
IT,123
DE,73
PL,55
US,52
FR,42
HR,28
CZ,26
ES,26
GR,26


Collect only CA sites:

In [60]:
ctyBks_CA = []
for i in range(num_of_networks):
  if ctyBks['networks'][i]['location']['country'] == 'CA':
    
    ctyBks_CA.append(ctyBks['networks'][i])


In [68]:
print(len(ctyBks_CA))
ctyBks_CA[1]

6


{'company': ['Motivate International, Inc.', 'PBSC Urban Solutions'],
 'gbfs_href': 'https://tor.publicbikesystem.net/ube/gbfs/v1/',
 'href': '/v2/networks/bixi-toronto',
 'id': 'bixi-toronto',
 'location': {'city': 'Toronto, ON',
  'country': 'CA',
  'latitude': 43.653226,
  'longitude': -79.3831843},
 'name': 'Bike Share Toronto'}

In [20]:
canadian_ntwrks = []
for i in range(num_of_networks):
  if ctyBks['networks'][i]['location']['country'] == 'CA':
    
    canadian_ntwrks.append(ctyBks['networks'][i])


In [72]:
print(len(canadian_ntwrks))
for i in range(6):
  print(canadian_ntwrks[i]['location']['city'])

6
Montréal, QC
Toronto, ON
Hamilton, ON
Vancouver
Québec
Saguenay


Still only 6.

### Put it all together into a function:

In [74]:
def ctyBksCityCountry_df(CCode):

  ctyBks = requests.get('https://api.citybik.es/v2/networks').json()
  num_of_networks = len(ctyBks['networks'])

  ctyBks_CCode = []
  for i in range(num_of_networks):
    if ctyBks['networks'][i]['location']['country'] == CCode:
  
      city = ctyBks['networks'][i]['location']['city']
      country = ctyBks['networks'][i]['location']['country']
      id = ctyBks['networks'][i]['id']
      
      ctyBks_CCode.extend([{'city': city, 'country': country, 'id': id}])

  ctyBks_CCode_sites = pd.DataFrame(ctyBks_CCode).sort_values('city')
  return ctyBks_CCode_sites

Try a Couple other Countries:

In [82]:
# I've always dreamed about travelling to Ireland!!
Ireland = ctyBksCityCountry_df('IE')
Ireland.head(5)

,city,country,id
0,Cork,IE,cork
3,Dublin,IE,dublinbikes
2,Galway,IE,galway
1,Limerick,IE,limerick


In [81]:
# I have family in Netherland:
Netherland = ctyBksCityCountry_df('NL')
Netherland.head(5)

,city,country,id
1,Dordrecht,NL,nextbike-dordrecht
0,Wageningen,NL,cykl


In [77]:
# Hercules is cool:
Greece = ctyBksCityCountry_df('GR')
Greece.head(5)

,city,country,id
25,Athens,GR,athens-bikes
11,Didymoteicho,GR,easybike-didymoteicho
9,Florina,GR,cyclopolis-florina
12,Glyfada,GR,easybike-glyfada
13,Igoumenitsa,GR,easybike-igoumenitsa


In [227]:
# Peek at the egocentric nutty Cousin:
USA = ctyBksCityCountry_df('US')
USA.head(5)

,city,country,id
1,"Ann Arbor, MI",US,arborbike
0,"Aspen, CO",US,we-cycle
43,"Atlanta, GA",US,relay-atlanta
2,"Austin, TX",US,austin
6,"Boise, ID",US,boise-greenbike


In [83]:
# If I traveled to Italy, I could eat well:
Italy = ctyBksCityCountry_df('IT')
Italy.head(5)

,city,country,id
99,Acquarica del Capo,IT,bicincitta-acquarica-del-capo
97,Acquaviva delle fonti,IT,bicincitta-acquaviva-delle-fonti
14,Alba,IT,alba
111,Alghero,IT,bicincitta-alghero
66,Andria,IT,andria


## Send a request to CityBikes for the city of your choice. 

Sticking with Canada.  Its a tie between Toronto and Vancouver:

In [120]:
dataSave = 0

def ctyBks_cityData(cityName):
  """
  input: Case insensitive string format of a city name, 
  output: [{dictionary}] from citybik.es API pertaining to the input city.
  """
  ctyBksCity = str(cityName).lower()

  ctyBks = requests.get('https://api.citybik.es/v2/networks').json()
  num_of_networks = len(ctyBks['networks'])

  cityData = []
  for i in range(num_of_networks):
    if ctyBksCity in ctyBks['networks'][i]['location']['city'].lower():
      cityData.append(ctyBks['networks'][i])
  
  return cityData


def ctyBks_cityStations(cityName):
  """
  input: Case insensitive string format of a city name,
  output: .json() get.request response from citybik.es API all stations to the input City.
  """
  cityStations = ctyBks_cityData(cityName)

  network_address = cityStations[0]['href']
  url = f'https://api.citybik.es{network_address}'
  stationData = requests.get(url).json()['network']['stations']

  return stationData

In [219]:
vanCtyBksData = ctyBks_cityData('vanCOUver')
vanCtyBksData

[{'company': ['Vanncouver Bike Share Inc',
   'CycleHop LLC',
   'City of Vancouver',
   'Smoove SAS'],
  'href': '/v2/networks/mobibikes',
  'id': 'mobibikes',
  'location': {'city': 'Vancouver',
   'country': 'CA',
   'latitude': 49.2827,
   'longitude': -123.1207},
  'name': 'Mobi'}]

In [218]:
ctyBksStns_Vanvouver = ctyBks_cityStations('vanCOUver')
print(f'Total number of bike Stations: {len(ctyBksStns_Vanvouver)}')
ctyBksStns_Vanvouver[0]

Total number of bike Stations: 241


{'empty_slots': 16,
 'extra': {'bank_card': False, 'slots': 35, 'status': 'online', 'uid': '0001'},
 'free_bikes': 19,
 'id': '7a19c49f486d7c0c02b3685d7b240448',
 'latitude': 49.262487,
 'longitude': -123.114397,
 'name': '0001 10th & Cambie',
 'timestamp': '2022-12-03T18:28:44.713000Z'}

In [113]:
ctyBks_cityData('ToroNTO')

[{'company': ['Motivate International, Inc.', 'PBSC Urban Solutions'],
  'gbfs_href': 'https://tor.publicbikesystem.net/ube/gbfs/v1/',
  'href': '/v2/networks/bixi-toronto',
  'id': 'bixi-toronto',
  'location': {'city': 'Toronto, ON',
   'country': 'CA',
   'latitude': 43.653226,
   'longitude': -79.3831843},
  'name': 'Bike Share Toronto'}]

In [213]:
ctyBksStns_Toronto = ctyBks_cityStations('ToroNTO')
print(f'Total number of bike Stations: {len(ctyBksStns_Toronto)}')
ctyBksStns_Toronto[0]

Total number of bike Stations: 655


{'empty_slots': 5,
 'extra': {'address': 'Queen St E / Woodward Ave',
  'altitude': 0.0,
  'ebikes': 0,
  'has_ebikes': True,
  'last_updated': 1670091762,
  'normal_bikes': 13,
  'payment': ['key', 'transitcard', 'creditcard', 'phone'],
  'payment-terminal': True,
  'renting': 1,
  'returning': 1,
  'slots': 19,
  'uid': '7303'},
 'free_bikes': 13,
 'id': 'fb337bbed72e2be090071e199899b2be',
 'latitude': 43.665269,
 'longitude': -79.319796,
 'name': 'Queen St E / Woodward Ave',
 'timestamp': '2022-12-03T18:27:11.540000Z'}

In [123]:
ctyBksStns_Toronto[1]

{'empty_slots': 7,
 'extra': {'address': 'Primrose Ave / Davenport Rd',
  'altitude': 0.0,
  'ebikes': 0,
  'has_ebikes': True,
  'last_updated': 1670056026,
  'normal_bikes': 7,
  'payment': ['key', 'transitcard', 'creditcard', 'phone'],
  'payment-terminal': True,
  'renting': 1,
  'returning': 1,
  'slots': 15,
  'uid': '7301'},
 'free_bikes': 7,
 'id': '4ff88d5880e71aa40d34cfe5d09b0ca7',
 'latitude': 43.67142,
 'longitude': -79.445947,
 'name': 'Primrose Ave / Davenport Rd',
 'timestamp': '2022-12-03T08:31:55.830000Z'}

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [199]:
def stationDataFrame(cityName):
  data = ctyBks_cityStations(cityName)
  df_list = []
  for i in data:
    stationData = {
      'name': i['name']
      , 'free_bikes': i['free_bikes']
      , 'empty_slots': i['empty_slots']
      , 'slots': i['extra']['slots']
      , 'altitude': i['extra']['altitude']
      , 'ebikes': i['extra']['ebikes']
      , 'has_ebikes': i['extra']['has_ebikes']
      , 'normal_bikes': i['extra']['normal_bikes']
      , 'payment': i['extra']['payment']
      , 'payment-terminal': i['extra']['payment-terminal']
      , 'renting': i['extra']['renting']
      , 'returning': i['extra']['returning']
      , 'uid': i['extra']['uid']
      , 'last_updated': i['extra']['last_updated']
      , 'address': i['extra']['address']
      , 'timestamp': i['timestamp']
      , 'latitude': i['latitude']
      , 'longitude': i['longitude']
      , 'id': i['id']
    }
    df_list.append(stationData)
  df = pd.DataFrame(df_list)

  # Preserve each unique timestamped df with timestamped saves:
  df.to_csv(f'../data/ctyBks_csv/df_ctybks_{cityName}({datetime.now()}).csv', index = False)
  return df


Put your parsed results into a DataFrame.

In [220]:
df = stationDataFrame('toronto')
df.head()

,name,free_bikes,empty_slots,slots,altitude,ebikes,has_ebikes,normal_bikes,payment,payment-terminal,renting,returning,uid,last_updated,address,timestamp,latitude,longitude,id
0,Queen St E / Woodward Ave,13,5,19,0.0,0,True,13,"[key, transitcard, creditcard, phone]",True,1,1,7303,1670092301,Queen St E / Woodward Ave,2022-12-03T18:32:24.317000Z,43.665269,-79.319796,fb337bbed72e2be090071e199899b2be
1,Primrose Ave / Davenport Rd,7,7,15,0.0,0,True,7,"[key, transitcard, creditcard, phone]",True,1,1,7301,1670092173,Primrose Ave / Davenport Rd,2022-12-03T18:32:24.323000Z,43.671420,-79.445947,4ff88d5880e71aa40d34cfe5d09b0ca7
2,Queen St. E / Rhodes Ave.,10,12,23,0.0,0,True,10,"[key, transitcard, creditcard, phone]",True,1,1,7309,1670092289,Queen St. E / Rhodes Ave.,2022-12-03T18:32:24.324000Z,43.666224,-79.317693,a09c67c0b419654d907c9134b108e328
3,Bond St / Queen St E,12,12,25,0.0,0,True,12,"[key, transitcard, creditcard, phone]",True,1,1,7045,1670092256,Bond St / Queen St E,2022-12-03T18:32:24.334000Z,43.653236,-79.376716,d6a9daee68070a8b106cfb598d81308c
4,Church St / Alexander St,0,12,15,0.0,0,True,0,"[key, transitcard, creditcard, phone]",True,1,1,7044,1670092301,Church St / Alexander St,2022-12-03T18:32:24.334000Z,43.663722,-79.380288,8f8af40d9388c8a3962559e8681d3db7


In [202]:
df.shape

(655, 19)

In [204]:
df.describe()

,free_bikes,empty_slots,slots,altitude,ebikes,normal_bikes,renting,returning,last_updated,latitude,longitude
count,655.000000,655.000000,655.000000,645.000000,655.000000,655.000000,655.0,655.0,6.550000e+02,655.000000,655.000000
mean,8.740458,9.421374,18.893130,0.173767,0.045802,8.694656,1.0,1.0,1.670066e+09,43.666004,-79.393227
std,6.739345,6.853837,6.273399,4.038028,0.578179,6.747627,0.0,0.0,7.871636e+01,0.028695,0.057671
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.670066e+09,43.588077,-79.545827
25%,3.000000,4.000000,15.000000,0.000000,0.000000,3.000000,1.0,1.0,1.670066e+09,43.648658,-79.425197
50%,7.000000,9.000000,19.000000,0.000000,0.000000,7.000000,1.0,1.0,1.670066e+09,43.660833,-79.393249
75%,13.000000,14.000000,22.000000,0.000000,0.000000,13.000000,1.0,1.0,1.670066e+09,43.676403,-79.369930
max,54.000000,44.000000,56.000000,102.080000,12.000000,54.000000,1.0,1.0,1.670066e+09,43.788319,-79.123505


In [223]:
import datetime as dt
from datetime import datetime

In [224]:
type(df['timestamp'][0])

str

In [225]:
print(df['timestamp'][100])
date, time = df['timestamp'][100].split('T')
print(date)
time, tz = time.split('.')
time = f'{time[:-3]}'
print(time)
print(tz)

2022-12-03T18:32:24.600000Z
2022-12-03
18:32
600000Z


In [226]:
df['timestamp'][100]

'2022-12-03T18:32:24.600000Z'